In [3]:
!pip install flask-ngrok
!pip install flask-bootstrap
!pip install pyngrok==4.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460118 sha256=7b07f60c94ad2cb90d16e0bc34beaa2b48ccc1cf01a0709da0069b6683562359
  Stored in directory: /root/.cache/pip/wheels/6f/33/ad/26540e84a28334e5dfeda756df270f95353779f03bc5cf40d4
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3926 sha256=70eb760efb1c5de2df126eb714e4beb9a5594ad3f6a1b5b8f8b8eea10e16d6f1
  Stored in directory: /root/.cache/pip/wheels/19/31/99/2ec5b4459cac4d801d6201d501a354366d180afc9f8bb2d333
Successfully built flask-bootstrap visitor
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=5b42d3e370c135d3246fdcde5a4496541dc5b671d7708d21034c86767f4b1cb2
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2e

In [4]:
import pandas as pd
from flask_ngrok import run_with_ngrok
from flask import request, jsonify, Flask
import random as r

In [6]:
!ngrok authtoken <insert your token>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
from torchvision.models import resnet50, ResNet50_Weights
import joblib
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
joblib.dump(model, 'model.pkl')
print("Model dumped!")

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 136MB/s]


Model dumped!


In [8]:
# Preprocess image
from torchvision import transforms
from PIL import Image
img = Image.open('/content/Machine-Learning-Model-to-API/statics/img.jpg')
tfms = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224),
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
img = tfms(img).unsqueeze(0)

In [41]:
import torch
import json
from flask import Flask, render_template
from PIL import Image
from io import BytesIO
import base64
import traceback

app = Flask(__name__,static_folder='static')
run_with_ngrok(app)

@app.route("/")
def hello():
    return render_template('hello.html')


@app.route('/input', methods=['GET', 'POST'])
def show_index():
    img_path = '/content/Machine-Learning-Model-to-API/statics/img.jpg'
    with open(img_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

    return render_template('index.html', img_data=encoded_string)

@app.route('/predict', methods=['GET','POST'])
def predict():
    if model:
        try:
            model.eval()
            with torch.no_grad():
                logits = model(img)
            preds = torch.topk(logits, k=5).indices.squeeze(0).tolist()
            labels_map = json.load(open('/content/Machine-Learning-Model-to-API/labels_map.txt'))
            labels_map = [labels_map[str(i)] for i in range(1000)]
            output ={}
            for idx in preds:
                label = labels_map[idx]
                prob = torch.softmax(logits, dim=1)[0, idx].item()
                print('{:<75} ({:.2f}%)'.format(label, prob*100))
                output[label] = '({:.2f}%)'.format(prob*100)
                output[prob] = '({:.2f}%)'.format(prob*100)
            # return jsonify({'prediction': str(output)})
            return render_template('predict.html', data =output)

        except:

            return jsonify({'trace': traceback.format_exc()})
    else:
        print ('Train the model first')
        return ('No model here to use')

if __name__ == '__main__':
    model = joblib.load("/content/Machine-Learning-Model-to-API/model.pkl") # Load "model.pkl"
    print ('Model loaded')

    app.run()

Model loaded
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5dc2-34-42-214-100.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [09/Jan/2024 17:12:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jan/2024 17:12:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Jan/2024 17:12:03] "POST /predict HTTP/1.1" 200 -


giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca           (45.03%)
lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens         (0.24%)
limousine, limo                                                             (0.20%)
soccer ball                                                                 (0.16%)
Walker hound, Walker foxhound                                               (0.16%)


INFO:werkzeug:127.0.0.1 - - [09/Jan/2024 17:12:10] "POST /input HTTP/1.1" 200 -
